In [19]:
#only for ipynb: 
%load_ext autoreload
%autoreload 2
from importlib import reload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
PATH = "/home/kim2712/Desktop/research/rlabc/"
DEVICE="cuda"
import sys
sys.path.insert(0,"..")
sys.path.insert(0,PATH)
import numpy as np
import os

from IPython.core.debugger import set_trace

from rlabc._data import decoder
import numpy as np
import torch
import matplotlib.pyplot as plt

from rlabc.othermethods import abc
from rlabc import tree_abc_JK

#PATH = "/Users/seanohagan/projects/rl-abc/"

import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

import os
import sys
sys.path.insert(0, "/home/kim2712/Desktop/research/generative_quantile/generative_qunatile")
sys.path.insert(0, "/home/kim2712/Desktop/research/rlabc")

from _data.gaussian_conjugate import posterior_sampler
from brenier.models import ConditionalConvexQuantile#,dual_JK               
from _utils.toy_2d_util import test

from _data.simulator_gaussian import Conjugate_Model
import pickle
from rlabc import treeproposal

In [21]:
n_samples=2
from config import parse_args
args = parse_args("",None)   
HPARAM = {"nu":args.nu, "sigma0_sq":args.sigma0_sq, 
          "mu0":args.mu0,"kappa":args.kappa}

import pathlib
pathlib.Path("./data").mkdir(parents=True, exist_ok=True)

In [22]:
#for M in [-4.1, -3.1, -2.1, -1.1, -.1, 0.5, 1.2, 2.5, +3.1]:
M=0.1
data=np.array([M for _ in range(n_samples)])
    
model = Conjugate_Model(theta_true=np.array([-4.7956, 2.4520]),
                    input_dim=2,
                    h_param=HPARAM,
                    obs_data=data)

rabc = abc.ABCR(10000, 
                model.support, 
                model.prior_sim, 
                model.discrepancy, 
                model.observed_data)
eps_init = np.quantile(rabc.discrepancies, 0.2)
eps_sched = eps_init * (0.9 ** np.arange(100))


tabc = tree_abc_JK.minibatch_TreeABC(100000, model.support, model.prior_sim, 
                        model.discrepancy, 
                        model.observed_data, 
                        model.prior_density, 
                        eps_sched, 
                        verbose=True,
                        proposal_update_rule=treeproposal.RULE_OPTIMAL_MAX_A, #treeproposal.RULE_POSTERIOR, 
                        n_leaves=1000)

  0%|          | 213/100000 [00:00<00:47, 2122.74it/s]

MPTF: None


  5%|▍         | 4716/100000 [00:01<00:33, 2833.54it/s]/home/kim2712/Desktop/research/rlabc/rlabc/treeproposal.py:204: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(np.where(pi != 0, q*p /pi, 0)) / np.sum(np.where(q != 0, pi*p/q, 0))


Batch 1 done, 4724 sims done. Current epsilon = 0.8049029710733843
49


 22%|██▏       | 21514/100000 [00:27<02:00, 652.99it/s]/home/kim2712/Desktop/research/rlabc/rlabc/treeproposal.py:204: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(np.where(pi != 0, q*p /pi, 0)) / np.sum(np.where(q != 0, pi*p/q, 0))
 22%|██▏       | 21580/100000 [00:27<02:37, 496.43it/s]

...

Batch 6 done, 96190 sims done. Current epsilon = 0.47528715538912275


/home/kim2712/Desktop/research/rlabc/rlabc/treeproposal.py:204: RuntimeWarning: invalid value encountered in true_divide
  return np.sum(np.where(pi != 0, q*p /pi, 0)) / np.sum(np.where(q != 0, pi*p/q, 0))
 96%|█████████▋| 96332/100000 [02:19<00:09, 373.16it/s]

56


100%|██████████| 100000/100000 [02:25<00:00, 688.13it/s]

Batch 7 done, 100000 sims done. Current epsilon = 0.42775843985021045


In [ ]:
import pickle

path = F"./data/tabc_M{M}.dat"

with open(path, 'wb') as file:
    pickle.dump(tabc, file)
    

In [24]:
np.save(F"./data/tabc_M{M}.npy",tabc.posteriors[-1])

In [31]:
self.massive_sample(batch_size=100, n_proposal=5000)

(array([[-0.22742969,  0.63349879],
        [ 0.11778092,  1.43749557],
        [-0.42639701,  0.64879357],
        [-0.07070769,  1.48005759],
        [ 0.60862419,  0.87799127],
        [-0.85730714,  1.09045058],
        [-0.6597556 ,  1.33866988],
        [-0.98063446,  2.29198606],
        [ 0.30575105,  0.68505789],
        [-0.07070769,  1.48005759],
        [-0.07070769,  1.48005759],
        [-0.6597556 ,  1.33866988],
        [ 0.30575105,  0.68505789],
        [ 1.05976397,  1.02243182],
        [-0.59400099,  0.48296207],
        [ 0.27964084,  0.97317412],
        [-0.42639701,  0.64879357],
        [ 0.34122164,  0.84432095],
        [-0.07070769,  1.48005759],
        [ 0.27964084,  0.97317412],
        [-0.85730714,  1.09045058],
        [ 0.34122164,  0.84432095],
        [-0.6597556 ,  1.33866988],
        [-0.22742969,  0.63349879],
        [-0.42639701,  0.64879357],
        [ 0.56477866,  0.64817608],
        [ 0.2168036 ,  0.70499605],
        [-0.22742969,  0.633

In [26]:
self= tabc
self.tree_model

DecisionTreeClassifier(max_depth=6, max_leaf_nodes=1000, min_samples_leaf=10)

In [27]:
self.tree_obj

In [28]:
self.TP 